In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy.stats import randint
low, high, RVsize, numRV = 0, 5, 1000, 4
rv = np.split(randint.rvs(low, high, size=RVsize*(numRV - 1)), numRV - 1)
rv = np.append(rv, np.remainder(np.sum(rv, axis=0), high)[None,:], axis=0)
histRV = np.array(np.unique(rv[0], axis=0, return_counts=True)[1])[None,:]
histRV2 = np.array(np.unique(rv[(0,1),:], axis=1, return_counts=True)[1])[None,:]
for i in range(1, numRV):
    histRV = np.append(histRV, np.array(np.unique(rv[i], axis=0, return_counts=True)[1])[None,:], axis=0)
    if i!=1:
        for j in range(i):
            histRV2 = np.append(histRV2, np.array(np.unique(rv[(j,i),:], axis=1, return_counts=True)[1])[None,:], axis=0)
pmfRV = histRV/RVsize
pmfRV2 = histRV2/RVsize
H = -np.average(np.log(pmfRV), weights=pmfRV, axis=1)
H2 = -np.average(np.log(pmfRV2), weights=pmfRV2, axis=1)
MI = np.zeros(H2.shape)
index = 0;
for i in range(1, numRV):
    for j in range(i):
        MI[index] = H2[index] - H[i] - H[j]
        index = index + 1

In [37]:
from scipy.special import comb
def subset(size, index):
    subset = [-1]
    sum = 0
    for numOutput in range(size):
        c = comb(size, numOutput)
        if index >= sum + c:
            sum += c
        else:
            break
    #print (numOutput)
    numLeft = numOutput
    for candidate in range(size-1, -1, -1):
        if index == sum:
            for remaining in range(numLeft-1, -1, -1):
                if subset[0] == -1:
                    subset[0] = remaining
                else:
                    subset = np.append(subset, remaining)
            break
        elif 0 == numLeft:
            break
        elif (index - sum) >= comb(candidate, numLeft):
            sum += comb(candidate, numLeft)
            if subset[0] == -1:
                subset[0] = candidate
            else:
                subset = np.append(subset, candidate)
            numLeft -= 1
    #print(output)
    if subset[0] != -1:
        return subset
    

In [41]:
print(subset(5, 16))

[2 1 0]


In [48]:
def ConditionSet(size, Resp, index):
    set = subset(size - 1, index)
    cond = [-1]
    for element in set:
        if element >= Resp:
            element += 1
        if cond[0] == -1:
            cond[0] = element
        else:
            cond = np.append(cond, element)
    return cond

In [81]:
def DiscreteEntropy(y):
    pmf = np.unique(y, return_counts=True, axis=y.ndim-1)[1]/y.size
    return -np.average(np.log(pmf), weights=pmf)

In [84]:
def CondEntropyScorer(estimator, X, y):
    y_est = estimator.predict(X)
    return DiscreteEntropy(np.array([y,y_est])) - DiscreteEntropy(y_est)
    #pmfY_est = np.unique(y_est, return_counts=True)[1]/y_est.size
    #HY_est = -np.average(np.log(pmfY_est), weights=pmfY_est)
    #pmfYY = np.unique([y,y_est], return_counts=True, axis=1)[1]/y_est.size
    #HYY = -np.average(np.log(pmfYY), weights=pmfYY)
    #diff = np.abs(y-y_est).max()
    #return HYY - HY_est

In [85]:
from sklearn import tree
from sklearn.model_selection import cross_val_score
#print(np.transpose(rv[ConditionSet(numRV, 0, 6)]))
#print(rv[0])
clf = tree.DecisionTreeClassifier()
#clf = clf.fit(np.transpose(rv[ConditionSet(numRV, 0, 6)]), rv[0])
#print (np.transpose(rv[ConditionSet(numRV, 0, 6), 0][:, None]))
#clf.predict(np.transpose(rv[ConditionSet(numRV, 0, 6), 0][:, None]))
print (cross_val_score(clf,np.transpose(rv[ConditionSet(numRV, 0, 6)]), rv[0], cv=3, scoring=CondEntropyScorer))

[2.26359779 2.2577532  2.28464841]


In [87]:
numComb = np.power(2, numRV - 1)
Entropy = np.zeros((numRV, numComb))
CVFold = 3
for Resp in range(numRV):
    Entropy[Resp,0] = DiscreteEntropy(rv[Resp])
    for sI in range(1, numComb):
        Entropy[Resp,sI] = np.mean(cross_val_score(clf,np.transpose(rv[ConditionSet(numRV, Resp, sI)]), rv[Resp], cv=CVFold, scoring=CondEntropyScorer))

In [88]:
Entropy

array([[1.60366556, 2.27492819, 2.27108138, 2.28797665, 2.27190833,
        2.27041932, 2.26866647, 2.27190833],
       [1.6060722 , 2.27554816, 2.28578606, 2.29361751, 2.27179159,
        2.26736387, 2.27192031, 2.27179159],
       [1.60779162, 2.27508102, 2.28774582, 2.29056029, 2.27197619,
        2.27237191, 2.27893154, 2.27197619],
       [1.60873656, 2.287451  , 2.28560451, 2.2841897 , 2.28492286,
        2.27758752, 2.2706059 , 2.28492286]])